## Exercise  8  |  data  in  f2ex8  |  Using  slopes  (declives.tif),  slope  exposure  (exposicao.tif),  and distance to watercourses (water.shp), produce a raster with the following characteristics.

In [ ]:
slopes = '/mnt/d/cwsig/exercicios_2023_24/f2ex8/declives.tif'

aspect = '/mnt/d/cwsig/exercicios_2023_24/f2ex4/exporcls.tif'

water = '/mnt/d/cwsig/exercicios_2023_24/f2ex8/water.shp'

output = '/mnt/d/cwsig/exercicios_2023_24/f2ex8/resrst3.tif'

In [ ]:
import os

from glass.pys.oss import mkdir, fprop
from glass.pys.tm import now_as_str
from glass.wenv.grs import run_grass
from glass.prj import proj
from glass.prop.prj import get_epsg

In [ ]:
ws, loc = os.path.dirname(output), now_as_str(utc=True)

gb = run_grass(ws, location=loc, srs=slopes)

# Start grass gis session
import grass.script.setup as gsetup
gsetup.init(gb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.it.shp import shp_to_grs
from glass.rst.dst import grow_distance
from glass.dtt.rst.torst import grsshp_to_grsrst
from glass.rst.alg import grsrstcalc
from glass.rst.local import combine
from glass.prop.rst import raster_report
from glass.rst.rcls.grs import grs_rcls, rcls_rules

In [ ]:
shpepsg = get_epsg(water)
refepsg = get_epsg(slopes)

if refepsg != shpepsg:
    water = proj(water, os.path.join(ws, loc, os.path.basename(water)), refepsg, api="ogr2ogr")

In [ ]:
# import data
gslope = rst_to_grs(slopes)
gaspec = rst_to_grs(aspect)
gwater = shp_to_grs(water)

# Water to Raster
rst_water = grsshp_to_grsrst(gwater, 1, f"rst_{gwater}")

# Calculate grow distance
dwater = grow_distance(rst_water, 'dwater', api="grass")

# Reclassify rasters
rcls_slope = grsrstcalc(f"if({gslope} <= 10, 1, 0)", f'rcls_{gslope}')
rcls_aspec = grsrstcalc((
    f"if({gaspec} == 5 || {gaspec} == 6 "
    f"|| {gaspec} == 7, 1, 0)"
), f"rcls_{gaspec}")

rcls_dist = grsrstcalc(f"if({dwater} < 200, 1, 0)", f'rcls_{dwater}')

# Combine
cmb = combine([rcls_slope, gaspec, rcls_dist], 'rst_combine', api="grass")

# Reclassify combine
cmbtxt = raster_report(cmb, os.path.join(ws, loc, 'combine_report.txt'), ascmd=True)

In [ ]:
# Combine values
otxt = open(cmbtxt, 'r')

dcmb = {}
c = 0
for l in otxt.readlines():
    try:
        if c >= 4:
            pl = l.split('|')

            rval = pl[1]

            if rval == '*':
                continue

            cats = pl[2].split('; ')

            catsv = []
            for cat in cats:
                _cat = cat.split(' ')[1]

                catsv.append(int(_cat))

            dcmb[int(rval)] = catsv

        c += 1
    except: break

In [ ]:
rules = {}

for v in dcmb:
    # See if it is one
    if dcmb[v][0] == 1 and dcmb[v][1] == 6 and dcmb[v][2] == 1:
        rules[v] = 1

    elif dcmb[v][0] == 1 and dcmb[v][1] == 5 and dcmb[v][2] == 1:
        rules[v] = 2

    elif dcmb[v][0] == 1 and dcmb[v][1] == 7 and dcmb[v][2] == 1:
        rules[v] = 3

    else:
        rules[v] = 4

rcls_file = rcls_rules(rules, os.path.join(ws, loc, 'rclsrules.txt'))

out = grs_rcls(cmb, rcls_file, fprop(output, 'fn'), as_cmd=True)

grs_to_rst(out, output, as_cmd=True, rtype=int)